In [ ]:
using Pkg; Pkg.activate("truncatedcg")

# Trust region with truncated conjugate gradient

$$ \min g_k^T d + \tfrac{1}{2}d^TB_kd \qquad \|d\| \leq \Delta. $$

In [ ]:
using Plots, Krylov, LinearAlgebra
gr(size=(600,600))

In [ ]:
B = [10.0 1.0;  # définie positive !
      1.0 2.0]
g = -B * ones(2)

m(d) = dot(d, g) + dot(d, B * d) / 2  # quadratic model

contour(-0.5:0.05:2, -0.5:0.05:2, (x, y) -> m([x; y]), leg=false)

scatter!([0.0; 1.0], [0.0; 1.0], c=:black, ms=3)

dG = -g * dot(g, g) / dot(g, B * g)  # gradient step
plot!([0.0; dG[1]], [0.0; dG[2]], c=:blue, l=:arrow)

dcg = cg(B, -g)[1]  # Newton step: solve Bd = -g
plot!([0.0; dcg[1]], [0.0; dcg[2]], c=:red, l=:arrow)

xlims!(-0.5, 2)
ylims!(-0.5, 2)

The Gay-Sorensen theorem says that if $d$ is the global minimizer of the trust-region subproblem, there exists $\lambda \geq 0$ such that $$(B + \lambda I) d = -g.$$

In [ ]:
dsol(λ) = cg(B + λ * I, -g)[1]
D = hcat([dsol(λ) for λ = 10.0 .^ (-6:0.1:6)]...)

contour(-0.5:0.05:2, -0.5:0.05:2, (x,y)->m([x;y]), leg=false)

dG = -g * dot(g, g) / dot(g, B * g)
scatter!([0.0; dG[1]; 1.0], [0.0; dG[2]; 1.0], c=:black, ms=3)

plot!(D[1,:], D[2,:], c=:blue)

xlims!(-0.5, 2)
ylims!(-0.5, 2)

Let's visualize the truncated CG step as a function of the trust-region radius $\Delta$.

In [ ]:
using Interact

@manipulate for Δ = 0.0:0.01:1.5
    contour(-0.5:0.05:2, -0.5:0.05:2, (x,y) -> norm([x;y]) > Δ ? NaN : m([x;y]), leg=false)

    t = range(0, stop=2π, length=100)
    plot!(Δ * cos.(t), Δ * sin.(t), c=:green, l=:dash)

    dG = -g * dot(g, g) / dot(g, B * g)
    scatter!([0.0; dG[1]; 1.0], [0.0; dG[2]; 1.0], c=:black, ms=3)

    d = cg(B, -g, radius=Δ)[1]
    plot!([0.0; d[1]], [0.0; d[2]], c=:red, l=:arrow)

    xlims!(-0.5, 2)
    ylims!(-0.5, 2)
end

In [ ]:
# Si vous avez un WebIO une erreur
using Conda
Conda.pip_interop(true)
Conda.pip("install", "webio_jupyter_extension")